# Lab 03: LSTM with Pytorch - solution

In [ ]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/CS5242_2025_codes/labs_lecture06/lab03_lstm_pytorch'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import utils

### With or without GPU?

It is recommended to run this code on GPU:<br> 
* Time for 1 epoch on CPU : 274 sec ( 4.56 min)<br> 
* Time for 1 epoch on GPU : 10.1 sec w/ GeForce GTX 1080 Ti <br>

In [3]:
device= torch.device("cuda")
# device= torch.device("cpu")
print(device)

if torch.cuda.is_available():
    print('cuda available with GPU:',torch.cuda.get_device_name(0))

cuda
cuda available with GPU: NVIDIA RTX A5000


### Download Penn Tree Bank

The tensor train_data consists of 20 columns of 46,479 words.<br>
The tensor test_data consists of 20 columns of 4,121 words.

In [4]:
from utils import check_ptb_dataset_exists
data_path=check_ptb_dataset_exists()

train_data  =  torch.load(data_path+'ptb/train_data.pt')
test_data   =  torch.load(data_path+'ptb/test_data.pt')

print(  train_data.size()  )
print(  test_data.size()   )

torch.Size([46479, 20])
torch.Size([4121, 20])


### Some constants associated with the data set

In [5]:
bs = 20

vocab_size = 10000


### Make a LTSM class

Step 1: Embed the discrete words into a continuous space using PyTorch’s nn.Embedding().

Step 2: Process the embedded words with a LSTM layer using PyTorch’s nn.LSTM().

Step 3: Project the processed tokens back into the vocabulary space by outputting a score vector over the vocabulary set with PyTorch’s nn.Linear().



In [6]:
class three_layer_recurrent_net(nn.Module):

    def __init__(self, hidden_size):
        super(three_layer_recurrent_net, self).__init__()
        
        self.layer1 = nn.Embedding( vocab_size  , hidden_size  )
        self.layer2 = nn.LSTM(      hidden_size , hidden_size  )
        self.layer3 = nn.Linear(    hidden_size , vocab_size   )

        
    def forward(self, word_seq, h_init, c_init ):
        
        g_seq                      =   self.layer1( word_seq )  
        h_seq , (h_final,c_final)  =   self.layer2( g_seq , (h_init,c_init) )      
        score_seq                  =   self.layer3( h_seq )
        
        return score_seq,  h_final , c_final


### Build the net. Choose the hidden size to be 300. How many parameters in total?

In [8]:
hidden_size=300

net = three_layer_recurrent_net( hidden_size )

print(net)

utils.display_num_param(net)

three_layer_recurrent_net(
  (layer1): Embedding(10000, 300)
  (layer2): LSTM(300, 300)
  (layer3): Linear(in_features=300, out_features=10000, bias=True)
)
There are 6732400 (6.73 million) parameters in this neural network


### Send the weights of the networks to the GPU

In [9]:
net = net.to(device)

### Set up manually the weights of the embedding module and Linear module

In [10]:
net.layer1.weight.data.uniform_(-0.1, 0.1)

net.layer3.weight.data.uniform_(-0.1, 0.1)

print('')

### Choose the criterion, as well as the following important hyperparameters: 
* initial learning rate = 5
* sequence length = 35

In [11]:
criterion = nn.CrossEntropyLoss()

my_lr = 5

seq_length = 35

### Function to evaluate the network on the test set

In [12]:
def eval_on_test_set():

    running_loss=0
    num_batches=0    
       
    h = torch.zeros(1, bs, hidden_size)
    c = torch.zeros(1, bs, hidden_size)
   
    h=h.to(device)
    c=c.to(device)
       
    for count in range( 0 , 4120-seq_length ,  seq_length) :
               
        minibatch_data =  test_data[ count   : count+seq_length   ]
        minibatch_label = test_data[ count+1 : count+seq_length+1 ]
        
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
                                  
        scores, h, c  = net( minibatch_data, h , c)
        
        minibatch_label =   minibatch_label.view(  bs*seq_length ) 
        scores          =            scores.view(  bs*seq_length , vocab_size)
        
        loss = criterion(  scores ,  minibatch_label )    
        
        h=h.detach()
        c=c.detach()
            
        running_loss += loss.item()
        num_batches += 1        
    
    total_loss = running_loss/num_batches 
    print('test: exp(loss) = ', math.exp(total_loss)  )
        

### Do 8 passes through the training set.

In [14]:
start=time.time()

for epoch in range(8):
    
    # divide the learning rate by 3 except after the first epoch
    if epoch >= 2:
        my_lr = my_lr / 3
    
    # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # set the running quatities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
       
    # set the initial h and c to be the zero vector
    h = torch.zeros(1, bs, hidden_size)
    c = torch.zeros(1, bs, hidden_size)

    # send them to the gpu    
    h=h.to(device)
    c=c.to(device)
    
    for count in range( 0 , 46478-seq_length ,  seq_length):
        
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_data =  train_data[ count   : count+seq_length   ]
        minibatch_label = train_data[ count+1 : count+seq_length+1 ]        
        
        # send them to the gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # Detach to prevent from backpropagating all the way to the beginning
        # Then tell Pytorch to start tracking all operations that will be done on h and c
        h=h.detach()
        c=c.detach()
        h=h.requires_grad_()
        c=c.requires_grad_()
                       
        # forward the minibatch through the net        
        scores, h, c  = net( minibatch_data, h , c)
        
        # reshape the scores and labels to huge batch of size bs*seq_length
        scores          =            scores.view(  bs*seq_length , vocab_size)  
        minibatch_label =   minibatch_label.view(  bs*seq_length )       
        
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(  scores ,  minibatch_label )
        
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward()

        # do one step of stochastic gradient descent: R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        utils.normalize_gradient(net)
        optimizer.step()   
            
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
        
        
        
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))
    eval_on_test_set() 



epoch= 0 	 time= 8.83834171295166 	 lr= 5 	 exp(loss)= 279.9687988745503
test: exp(loss) =  178.10868726757766

epoch= 1 	 time= 17.781137228012085 	 lr= 5 	 exp(loss)= 127.68048614444065
test: exp(loss) =  134.52417483744682

epoch= 2 	 time= 26.72772240638733 	 lr= 1.6666666666666667 	 exp(loss)= 81.64102730697371
test: exp(loss) =  115.14347191842958

epoch= 3 	 time= 35.65802621841431 	 lr= 0.5555555555555556 	 exp(loss)= 67.26879812855505
test: exp(loss) =  111.17021786580035

epoch= 4 	 time= 44.56057024002075 	 lr= 0.1851851851851852 	 exp(loss)= 62.39634262579738
test: exp(loss) =  109.77064114639089

epoch= 5 	 time= 53.49890613555908 	 lr= 0.0617283950617284 	 exp(loss)= 60.64411339095102
test: exp(loss) =  109.00978534078715

epoch= 6 	 time= 62.42774677276611 	 lr= 0.0205761316872428 	 exp(loss)= 60.0067971359295
test: exp(loss) =  108.48025497327858

epoch= 7 	 time= 71.34986758232117 	 lr= 0.006858710562414266 	 exp(loss)= 59.77650121506264
test: exp(loss) =  108.1816265

### Choose one sentence (taken from the test set)

In [15]:
# SENTENCES FROM TEST SET

sentence1 = "some analysts expect oil prices to remain relatively"

sentence2 = "over the next days and weeks they say investors should look for stocks to"

sentence2 = "some analysts expect oil prices to remain relatively"

sentence3 = "prices averaging roughly $ N a barrel higher in the third"

sentence4 = "i think my line has been very consistent mrs. hills said at a news"

sentence5 = "this appears particularly true at gm which had strong sales in"

# OR MAKE YOUR OWN SENTENCE. 
# NO CAPITAL LETTER ALLOWED. EACH WORD MUST BE IN THE ALLOWED VOCABULARY OF 10,000 WORDS

sentence6= "he was very"


# CHOOSE THE SENTENCE
mysentence = sentence3

### Convert the sentence into a vector, then send to GPU

In [16]:
minibatch_data=utils.sentence2vector(mysentence)
      
minibatch_data=minibatch_data.to(device)

print(minibatch_data)

tensor([[1786],
        [8705],
        [3246],
        [ 416],
        [  27],
        [  35],
        [2664],
        [ 209],
        [ 108],
        [  32],
        [3017]], device='cuda:0')


### Set the initial hidden state to zero, then run the LSTM.

In [17]:
h = torch.zeros(1, 1, hidden_size)
c = torch.zeros(1, 1, hidden_size)
h=h.to(device)
c=c.to(device)

scores , h, c = net(minibatch_data , h, c)

### Display the network prediction for the next word

In [18]:
print(mysentence, '... \n')

utils.show_next_word(scores)

prices averaging roughly $ N a barrel higher in the third ... 

83.3%	 quarter
2.5%	 and
1.7%	 <eos>
1.1%	 consecutive
1.0%	 than
0.9%	 world
0.9%	 day
0.8%	 period
0.5%	 but
0.4%	 game
0.4%	 term
0.4%	 week
0.4%	 market
0.3%	 month
0.3%	 session
0.2%	 area
0.2%	 quarters
0.2%	 range
0.2%	 floor
0.1%	 year
0.1%	 sector
0.1%	 tax
0.1%	 half
0.1%	 <unk>
0.1%	 fourth
0.1%	 fiscal
0.1%	 third
0.1%	 in
0.1%	 parties
0.1%	 case
